# [Demystifying Tensorflow Time Series: Local Linear Trend](https://towardsdatascience.com/demystifying-tensorflow-time-series-local-linear-trend-9bec0802b24a)

There was a nice, robust walkthrough of some of the Tensorflow structural time series functionality. I already want to explore this library and it unpacks a few concepts along the way. This notebook is just a companion to the above linked article that implements the presented material.

In [9]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import ipywidgets as ipw
from dataclasses import dataclass
from typing import Tuple, Optional
import panel as pn
from panel.interact import interact
from panel.widgets import FloatSlider
import param
import plotly.graph_objects as go

pn.extension("plotly")

## The Basic Local Linear Trend (LLT) Model

The LLT model allows both the slope $\beta$ and level $\mu$ in a series $y$ to vary over time, but only with random perturbations $\epsilon$ that are neither time-dependent nor correlated with each other.

\begin{align}
    \beta_t &= \beta_{t-1} + \epsilon_1 \\
    \mu_t &= \mu_{t-1} + \beta_{t-1} + \epsilon_2 \\
    y_t &= \mu_t + \epsilon_3
\end{align}

Both $\beta$ and $\mu$ are latent variables while $y$ is observed. The noise parameters $\epsilon$ are the parameters to be estimated. They are normally distributed:

\begin{align}
    \epsilon_1 &\sim N(0, \sigma_{\text{slope}}^2) \\
    \epsilon_2 &\sim N(0, \sigma_{\text{level}}^2) \\
    \epsilon_3 &\sim N(0, \sigma_{\text{obs}}^2) \\
\end{align}

In [31]:
class BasicLLTSim:
    
    def __init__(
        self, 
        e_slope: tfd.Distribution, 
        e_level: tfd.Distribution, 
        e_obs: tfd.Distribution,
        n: int = 100,
        start_vals: Tuple[float, float, float] = (0., 0., 0.)
    ) -> None:
        self.e_slope: tfd.Distribution = e_slope
        self.e_level: tfd.Distribution = e_level
        self.e_obs: tfd.Distribution = e_obs
        self.n: int = n
        self.start_vals: Tuple[float, float, float] = start_vals
        self.slopes: tf.Tensor = self.slope_series(n, start_vals[0])
        self.levels: tf.Tensor = self.level_series(n, start_vals[1], self.slopes)
        self.obs: tf.Tensor = self.obs_series(n, start_vals[2], self.levels)
            
    def slope_series(self, n: int, start_val: float) -> tf.Tensor:
        diffs: tf.Tensor = self.e_slope.sample(n)
        series: tf.Tensor = tf.cumsum(diffs) + start_val
        return series
    
    def level_series(self, n: int, start_val: float, slopes: tf.Tensor) -> tf.Tensor:
        series: List[float] = [tf.constant(start_val)]
        noise: tf.Tensor = self.e_level.sample(n)
        diffs: tf.Tensor = slopes + noise
        series: tf.Tensor = tf.cumsum(diffs) + start_val
        return series
    
    def obs_series(self, n: int, start_val: float, levels: tf.Tensor) -> tf.Tensor:
        diffs: tf.Tensor = self.e_obs.sample(n)
        series: tf.Tensor = levels + diffs
        return series
        
    def plot(self, figsize: Tuple[int, int] = (800, 500), **kwargs) -> Axes:
        fig: go.Figure = go.Figure()
        fig.add_trace(go.Scatter(
            x=list(range(self.obs.shape[0])), 
            y=self.obs,
            **kwargs
        ))
        title: str = f"e_slopes: {self.e_slope.parameters['scale']}," \
            +f"e_levels: {self.e_level.parameters['scale']}, e_obs: {self.e_obs.parameters['scale']}"
        fig.update_layout(
            title=title, 
            template="plotly_white",
            width=figsize[0],
            height=figsize[1],
            yaxis_range=[-1000,1000]
        )
        return fig
    
    def plot_components(self, figsize: Tuple[int, int] = (800, 500), **kwargs) -> Axes:
        series: Dict[str, tf.Tensor] = {
            "slopes": self.slopes,
            "levels": self.levels,
            "obs": self.obs
        }
        fig: go.Figure = go.Figure()
        for ser in series:
            fig.add_trace(go.Scatter(
                x=list(range(self.obs.shape[0])), 
                y=series[ser],
                name=ser,
                **kwargs
            ))
        title: str = f"e_slopes: {self.e_slope.parameters['scale']}," \
            +f"e_levels: {self.e_level.parameters['scale']}, e_obs: {self.e_obs.parameters['scale']}"
        fig.update_layout(
            title=title, 
            template="plotly_white",
            width=figsize[0],
            height=figsize[1]
        )
        return fig
    
    def plot_many(self, n: int = 10, figsize: Tuple[int, int] = (800, 500), **kwargs) -> Axes:
        fig: go.Figure = go.Figure()
        for i in range(n):
            new_bls: BasicLLTSim = BasicLLTSim(self.e_slope, self.e_level, self.e_obs)
            fig.add_trace(go.Scatter(
                x=list(range(new_bls.obs.shape[0])), 
                y=new_bls.obs,
                **kwargs
            ))
            
        title: str = f"e_slopes: {self.e_slope.parameters['scale']}," \
            +f"e_levels: {self.e_level.parameters['scale']}, e_obs: {self.e_obs.parameters['scale']}"
        fig.update_layout(
            title=title, 
            template="plotly_white",
            width=figsize[0],
            height=figsize[1]
        )
        return fig

bls: BasicLLTSim = BasicLLTSim(
    e_slope=tfd.Normal(0., 0.5),
    e_level=tfd.Normal(0., 0.1),
    e_obs=tfd.Normal(0., 1.)
)
    
bls.obs

<tf.Tensor: id=44028, shape=(100,), dtype=float32, numpy=
array([ -0.33326095,   0.86653304,   0.7560166 ,   2.3826427 ,
         0.8873035 ,   3.201775  ,   4.348256  ,   5.456853  ,
         4.295368  ,   6.8577714 ,   8.507946  ,   9.151389  ,
        12.361773  ,  13.182227  ,  13.1749    ,  14.862698  ,
        12.771112  ,  14.814399  ,  14.10454   ,  14.891189  ,
        15.862056  ,  14.782157  ,  15.209802  ,  16.39428   ,
        19.237282  ,  15.652312  ,  19.50612   ,  21.287922  ,
        19.395674  ,  25.49609   ,  23.737951  ,  26.785414  ,
        29.945698  ,  33.181896  ,  35.734596  ,  36.00598   ,
        37.88191   ,  40.178314  ,  41.463287  ,  42.486603  ,
        43.02979   ,  44.003277  ,  48.343033  ,  48.452118  ,
        47.766468  ,  49.61972   ,  50.00004   ,  50.83851   ,
        55.227257  ,  55.773335  ,  56.563988  ,  57.648045  ,
        59.553917  ,  60.246696  ,  62.70853   ,  63.015656  ,
        64.21048   ,  67.37908   ,  67.9598    ,  66.81002  

In [32]:
@interact(
    e_slope=FloatSlider(start=0., end=10., step=0.1, value=0.5), 
    e_level=FloatSlider(start=0., end=10., step=0.1, value=0.1), 
    e_obs=FloatSlider(start=0., end=10., step=0.1, value=1.)
)
def llt_sensitivity(e_slope:float, e_level: float, e_obs: float) -> Axes:
    print(e_slope)
    tmp_bls: BasicLLTSim = BasicLLTSim(
        e_slope=tfd.Normal(0., e_slope),
        e_level=tfd.Normal(0., e_level),
        e_obs=tfd.Normal(0., e_obs)
    )
    return tmp_bls.plot()
        
llt_sensitivity

0.5


Column
    [0] Column
        [0] FloatSlider(end=10.0, value=0.5)
        [1] FloatSlider(end=10.0, value=0.1)
        [2] FloatSlider(end=10.0, value=1.0)
    [1] Row
        [0] Plotly(Figure, name='interactive00251')

In [35]:
bls.plot_many()

In [34]:
@interact(sigma=FloatSlider(start=0., end=10., step=0.1, value=0.5))
def sigma_plot(sigma: float) -> Axes:
    series: tf.Tensor = tfd.Normal(0, sigma).sample(100)
    fig: go.Figure = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(series.shape[0])), 
        y=series
    ))
    return fig

sigma_plot

Column
    [0] Column
        [0] FloatSlider(end=10.0, value=0.5)
    [1] Row
        [0] Plotly(Figure, name='interactive00260')

In [ ]:
@interact(sigma=FloatSlider(start=0., end=10., step=0.1, value=0.5))
def sigma_data(sigma: float) -> tf.Tensor:
    series: tf.Tensor = tfd.Normal(0, sigma).sample(100)
    return series

sigma_data